Training a Conformal Prediction Model for House Price Range Estimation

Objective: Build a model to predict value intervals (prediction intervals) with a 90% coverage rate

Using a combination of LightGBM and ElasticNet with Conformal Prediction

Date: 2025/07/16

#### Import thư viện cần thiết

In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.linear_model import ElasticNet, Ridge
from sklearn.ensemble import RandomForestRegressor, HistGradientBoostingRegressor
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.ensemble import VotingRegressor
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.pipeline import make_pipeline
import joblib
import warnings
from datetime import date
warnings.filterwarnings('ignore')

#### Tải dữ liệu đã xử lý

In [2]:
print("Đang tải dữ liệu đã xử lý...")
X_train_scaled = np.load('processed/X_train_scaled.npy')
X_val_scaled = np.load('processed/X_val_scaled.npy')
X_test_scaled = np.load('processed/X_test_scaled.npy')
y_train = np.load('processed/y_train.npy')
y_val = np.load('processed/y_val.npy')

scaler = joblib.load('processed/scaler.pkl')
label_encoders = joblib.load('processed/label_encoders.pkl')
feature_columns = joblib.load('processed/feature_columns.pkl')

print(f"Kích thước X_train: {X_train_scaled.shape}")
print(f"Kích thước X_val: {X_val_scaled.shape}")
print(f"Kích thước X_test: {X_test_scaled.shape}")
print(f"Kích thước y_train: {y_train.shape}")
print(f"Kích thước y_val: {y_val.shape}")

Đang tải dữ liệu đã xử lý...
Kích thước X_train: (160000, 50)
Kích thước X_val: (40000, 50)
Kích thước X_test: (200000, 50)
Kích thước y_train: (160000,)
Kích thước y_val: (40000,)


#### Feature Engineering bổ sung

In [3]:
def create_advanced_features(X_scaled, feature_names):
    """
    Tạo các đặc trưng nâng cao để cải thiện MAE và RMSE
    """
    X_df = pd.DataFrame(X_scaled, columns=feature_names)
    current_year = date.today().year

    # Tỷ lệ diện tích
    if 'sqft' in X_df.columns and 'sqft_lot' in X_df.columns:
        X_df['sqft_ratio'] = X_df['sqft'] / (X_df['sqft_lot'] + 1)
        X_df['sqft_lot_log'] = np.log1p(np.abs(X_df['sqft_lot']))
        X_df['sqft_log'] = np.log1p(np.abs(X_df['sqft']))
    
    # Giá trị bất động sản
    if 'land_val' in X_df.columns and 'imp_val' in X_df.columns:
        X_df['total_val'] = X_df['land_val'] + X_df['imp_val']
        X_df['imp_land_ratio'] = X_df['imp_val'] / (X_df['land_val'] + 1)
        X_df['total_val_log'] = np.log1p(np.abs(X_df['total_val']))
        X_df['land_val_per_sqft'] = X_df['land_val'] / (X_df['sqft_lot'] + 1)
    
    # Đặc trưng nhà ở
    if 'beds' in X_df.columns and 'sqft' in X_df.columns:
        X_df['sqft_per_bed'] = X_df['sqft'] / (X_df['beds'] + 1)
        X_df['beds_sqft_interaction'] = X_df['beds'] * X_df['sqft']
    
    # Tổng số phòng tắm
    if 'bath_full' in X_df.columns and 'bath_3qtr' in X_df.columns and 'bath_half' in X_df.columns:
        X_df['total_bath'] = X_df['bath_full'] + X_df['bath_3qtr'] * 0.75 + X_df['bath_half'] * 0.5
        X_df['bath_per_bed'] = X_df['total_bath'] / (X_df['beds'] + 1)
    
    # Tuổi nhà và renovation
    if 'year_built' in X_df.columns:
        X_df['house_age'] = current_year - X_df['year_built']
        X_df['house_age_squared'] = X_df['house_age'] ** 2
        if 'year_reno' in X_df.columns:
            X_df['years_since_reno'] = np.where(X_df['year_reno'] > 0, current_year - X_df['year_reno'], X_df['house_age'])
    
    # Điểm view tổng hợp
    view_columns = [col for col in X_df.columns if col.startswith('view_')]
    if view_columns:
        X_df['total_view_score'] = X_df[view_columns].sum(axis=1)
        X_df['has_view'] = (X_df['total_view_score'] > 0).astype(int)
    
    # Đặc trưng garage
    if 'gara_sqft' in X_df.columns and 'sqft' in X_df.columns:
        X_df['garage_ratio'] = X_df['gara_sqft'] / (X_df['sqft'] + 1)
        X_df['has_garage'] = (X_df['gara_sqft'] > 0).astype(int)
    
    # Đặc trưng vị trí
    if 'latitude' in X_df.columns and 'longitude' in X_df.columns:
        X_df['lat_long_interaction'] = X_df['latitude'] * X_df['longitude']
        X_df['distance_from_center'] = np.sqrt((X_df['latitude'] - X_df['latitude'].mean())**2 + (X_df['longitude'] - X_df['longitude'].mean())**2)
    
    # Tương tác giữa grade và condition
    if 'grade' in X_df.columns and 'condition' in X_df.columns:
        X_df['grade_condition_interaction'] = X_df['grade'] * X_df['condition']
        X_df['quality_score'] = X_df['grade'] + X_df['condition']
    
    return X_df.values

print("Tạo đặc trưng nâng cao...")
X_train_enhanced = create_advanced_features(X_train_scaled, feature_columns)
X_val_enhanced = create_advanced_features(X_val_scaled, feature_columns)
X_test_enhanced = create_advanced_features(X_test_scaled, feature_columns)

print(f"Số đặc trưng sau khi tăng cường: {X_train_enhanced.shape[1]}")

Tạo đặc trưng nâng cao...
Số đặc trưng sau khi tăng cường: 72


#### Thiết lập mô hình cơ sở

In [4]:
print("Thiết lập mô hình tối ưu...")

# LightGBM với hyperparameters tối ưu
lgb_optimized = lgb.LGBMRegressor(
    objective='regression',
    metric='rmse',
    boosting_type='gbdt',
    num_leaves=127,
    learning_rate=0.03,
    feature_fraction=0.85,
    bagging_fraction=0.75,
    bagging_freq=0,
    min_child_samples=5,
    min_child_weight=0.1,
    max_depth=8,
    reg_alpha=0.0,
    reg_lambda=0.0,
    n_estimators=1000,
    random_state=42,
    n_jobs=-1,
    verbose=-1
)

# Hist Gradient Boosting tối ưu
gb_optimized = HistGradientBoostingRegressor(
    max_iter=600,
    learning_rate=0.05,
    max_depth=10,
    min_samples_leaf=20,
    max_leaf_nodes=127,
    max_bins=127,
    l2_regularization=0.01,
    random_state=42
)

Thiết lập mô hình tối ưu...


In [5]:
# Ensemble model với weights tối ưu
ensemble_optimized = VotingRegressor([
    ('lgb', lgb_optimized),
    ('hgb', gb_optimized)
], weights=[0.505, 0.495])

print("Đã thiết lập mô hình ensemble tối ưu (LGB + HistGB)")

Đã thiết lập mô hình ensemble tối ưu (LGB + HistGB)


#### Huấn luyện mô hình với Cross-Validation

In [6]:
print("Bắt đầu huấn luyện với K-Fold Cross-Validation...")

kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Đánh giá RMSE
cv_rmse_scores = []
cv_mae_scores = []

for fold, (train_idx, val_idx) in enumerate(kf.split(X_train_enhanced)):
    X_fold_train, X_fold_val = X_train_enhanced[train_idx], X_train_enhanced[val_idx]
    y_fold_train, y_fold_val = y_train[train_idx], y_train[val_idx]
    
    ensemble_optimized.fit(X_fold_train, y_fold_train)
    y_pred = ensemble_optimized.predict(X_fold_val)
    
    fold_rmse = np.sqrt(mean_squared_error(y_fold_val, y_pred))
    fold_mae = mean_absolute_error(y_fold_val, y_pred)
    
    cv_rmse_scores.append(fold_rmse)
    cv_mae_scores.append(fold_mae)
    
    print(f"Fold {fold+1} - RMSE: {fold_rmse:.2f}, MAE: {fold_mae:.2f}")

print(f"\nMean CV RMSE: {np.mean(cv_rmse_scores):.2f} ± {np.std(cv_rmse_scores):.2f}")
print(f"Mean CV MAE: {np.mean(cv_mae_scores):.2f} ± {np.std(cv_mae_scores):.2f}")

Bắt đầu huấn luyện với K-Fold Cross-Validation...
Fold 1 - RMSE: 102276.48, MAE: 56615.26
Fold 2 - RMSE: 104072.92, MAE: 56615.94
Fold 3 - RMSE: 101864.00, MAE: 56485.14
Fold 4 - RMSE: 100512.72, MAE: 56387.19
Fold 5 - RMSE: 102439.95, MAE: 56471.27

Mean CV RMSE: 102233.21 ± 1142.42
Mean CV MAE: 56514.96 ± 88.75


#### Huấn luyện mô hình cuối cùng

In [7]:
print("Huấn luyện mô hình cuối cùng...")

ensemble_optimized.fit(X_train_enhanced, y_train)

train_pred = ensemble_optimized.predict(X_train_enhanced)
val_pred = ensemble_optimized.predict(X_val_enhanced)

train_rmse = np.sqrt(mean_squared_error(y_train, train_pred))
train_mae = mean_absolute_error(y_train, train_pred)
val_rmse = np.sqrt(mean_squared_error(y_val, val_pred))
val_mae = mean_absolute_error(y_val, val_pred)

print(f"Training RMSE: {train_rmse:.2f}")
print(f"Training MAE: {train_mae:.2f}")
print(f"Validation RMSE: {val_rmse:.2f}")
print(f"Validation MAE: {val_mae:.2f}")

Huấn luyện mô hình cuối cùng...
Training RMSE: 68246.10
Training MAE: 43880.18
Validation RMSE: 102032.00
Validation MAE: 56072.10


#### Triển khai Conformal Prediction

In [8]:
def split_conformal_prediction(X_train, y_train, X_val, y_val, model, alpha=0.1):
    """
    Triển khai Split Conformal Prediction
    """
    model.fit(X_train, y_train)
    
    val_pred = model.predict(X_val)
    
    # Tính residuals
    residuals = np.abs(y_val - val_pred)
    
    # Tính quantile với điều chỉnh finite sample
    n_val = len(residuals)
    quantile_level = np.ceil((n_val + 1) * (1 - alpha)) / n_val
    quantile_level = min(quantile_level, 1.0)
    
    q_hat = np.quantile(residuals, quantile_level)
    
    # Tính MAE và RMSE của point predictions
    val_mae = mean_absolute_error(y_val, val_pred)
    val_rmse = np.sqrt(mean_squared_error(y_val, val_pred))
    
    return q_hat, val_mae, val_rmse

print("Áp dụng Split Conformal Prediction tối ưu...")
q_hat, conformal_mae, conformal_rmse = split_conformal_prediction(
    X_train_enhanced, y_train, X_val_enhanced, y_val,
    ensemble_optimized, alpha=0.1
)

print(f"Split Conformal quantile (q_hat): {q_hat:.2f}")
print(f"Conformal MAE: {conformal_mae:.2f}")
print(f"Conformal RMSE: {conformal_rmse:.2f}")

Áp dụng Split Conformal Prediction tối ưu...
Split Conformal quantile (q_hat): 128624.66
Conformal MAE: 56072.10
Conformal RMSE: 102032.00


#### Tạo khoảng dự đoán cho tập test

In [9]:
print("Tạo khoảng dự đoán cho tập test...")

test_pred = ensemble_optimized.predict(X_test_enhanced)

pi_lower = test_pred - q_hat
pi_upper = test_pred + q_hat

mean_width = np.mean(pi_upper - pi_lower)

print(f"Khoảng dự đoán Split Conformal - Mean width: {mean_width:.2f}")
print(f"Point predictions - Mean: {np.mean(test_pred):.2f}")
print(f"Point predictions - Std: {np.std(test_pred):.2f}")

Tạo khoảng dự đoán cho tập test...
Khoảng dự đoán Split Conformal - Mean width: 257249.33
Point predictions - Mean: 593226.63
Point predictions - Std: 410434.66


#### Đánh giá độ bao phủ trên tập validation

In [10]:
val_pred_final = ensemble_optimized.predict(X_val_enhanced)

val_pi_lower = val_pred_final - q_hat
val_pi_upper = val_pred_final + q_hat

coverage = np.mean((y_val >= val_pi_lower) & (y_val <= val_pi_upper))

print(f"Coverage Split Conformal: {coverage:.3f} (mục tiêu: 0.900)")

Coverage Split Conformal: 0.900 (mục tiêu: 0.900)


#### Tính Winkler Interval Score

In [11]:
def winkler_score(y_true, pi_lower, pi_upper, alpha=0.1):
    """
    Tính Winkler Interval Score
    """
    width = pi_upper - pi_lower
    
    lower_penalty = 2 * alpha * np.maximum(0, pi_lower - y_true)
    upper_penalty = 2 * alpha * np.maximum(0, y_true - pi_upper)
    
    score = width + lower_penalty + upper_penalty
    
    return score

val_winkler = winkler_score(y_val, val_pi_lower, val_pi_upper, alpha=0.1)
avg_winkler = np.mean(val_winkler)

print(f"Average Winkler Score: {avg_winkler:.2f}")

Average Winkler Score: 259654.24


#### Phân tích chi tiết về MAE và RMSE

In [12]:
print("PHÂN TÍCH CHI TIẾT MAE VÀ RMSE:")
print("="*50)

# Tính MAE và RMSE cho từng percentile
percentiles = [25, 50, 75, 90, 95]
for p in percentiles:
    threshold = np.percentile(y_val, p)
    mask = y_val <= threshold
    
    if np.sum(mask) > 0:
        mae_p = mean_absolute_error(y_val[mask], val_pred_final[mask])
        rmse_p = np.sqrt(mean_squared_error(y_val[mask], val_pred_final[mask]))
        
        print(f"Percentile {p}% (≤ ${threshold:,.0f}):")
        print(f"  MAE: {mae_p:,.2f}")
        print(f"  RMSE: {rmse_p:,.2f}")
        print(f"  Samples: {np.sum(mask):,}")

# Tính relative errors
relative_errors = np.abs(y_val - val_pred_final) / y_val
mape = np.mean(relative_errors) * 100

print(f"\nMAPE (Mean Absolute Percentage Error): {mape:.2f}%")
print(f"Median Absolute Error: {np.median(np.abs(y_val - val_pred_final)):,.2f}")

PHÂN TÍCH CHI TIẾT MAE VÀ RMSE:
Percentile 25% (≤ $304,088):
  MAE: 25,797.35
  RMSE: 46,262.88
  Samples: 10,000
Percentile 50% (≤ $455,000):
  MAE: 29,638.76
  RMSE: 48,931.88
  Samples: 20,073
Percentile 75% (≤ $725,000):
  MAE: 36,261.82
  RMSE: 59,499.44
  Samples: 30,107
Percentile 90% (≤ $1,100,000):
  MAE: 43,554.51
  RMSE: 72,117.76
  Samples: 36,118
Percentile 95% (≤ $1,425,000):
  MAE: 47,823.13
  RMSE: 80,362.35
  Samples: 38,001

MAPE (Mean Absolute Percentage Error): 9.91%
Median Absolute Error: 29,755.10


#### Lưu mô hình và kết quả

In [13]:
import os
os.makedirs('models', exist_ok=True)

joblib.dump(ensemble_optimized, 'models/optimized_ensemble_model.pkl')

conformal_params = {
    'q_hat': q_hat,
    'alpha': 0.1,
    'coverage': coverage,
    'mae': conformal_mae,
    'rmse': conformal_rmse
}
joblib.dump(conformal_params, 'models/optimized_conformal_params.pkl')

results = {
    'test_predictions': test_pred,
    'pi_lower': pi_lower,
    'pi_upper': pi_upper,
    'coverage': coverage,
    'winkler_score': avg_winkler,
    'mae': conformal_mae,
    'rmse': conformal_rmse,
    'train_mae': train_mae,
    'train_rmse': train_rmse,
    'val_mae': val_mae,
    'val_rmse': val_rmse,
    'cv_mae_mean': np.mean(cv_mae_scores),
    'cv_rmse_mean': np.mean(cv_rmse_scores)
}

np.save('models/optimized_results.npy', results)

print("Đã lưu tất cả mô hình và kết quả")

Đã lưu tất cả mô hình và kết quả


#### Tóm tắt kết quả cuối cùng

In [14]:
print("="*60)
print("KẾT QUẢ CUỐI CÙNG - SPLIT CONFORMAL PREDICTION")
print("="*60)

print(f"Mô hình: Optimized Ensemble (LGBM + HistGB)")
print(f"Số lượng features: {X_train_enhanced.shape[1]}")
print(f"Số lượng mẫu test: {len(test_pred):,}")
print()

print("PERFORMANCE METRICS:")
print(f"  Cross-Validation MAE: {np.mean(cv_mae_scores):,.2f} ± {np.std(cv_mae_scores):,.2f}")
print(f"  Cross-Validation RMSE: {np.mean(cv_rmse_scores):,.2f} ± {np.std(cv_rmse_scores):,.2f}")
print(f"  Training MAE: {train_mae:,.2f}")
print(f"  Training RMSE: {train_rmse:,.2f}")
print(f"  Validation MAE: {val_mae:,.2f}")
print(f"  Validation RMSE: {val_rmse:,.2f}")
print(f"  MAPE: {mape:.2f}%")
print()

print("CONFORMAL PREDICTION RESULTS:")
print(f"  Coverage thực tế: {coverage:.1%}")
print(f"  Average Winkler Score: {avg_winkler:,.2f}")
print(f"  Độ rộng khoảng trung bình: {mean_width:,.2f}")
print(f"  Quantile threshold (q_hat): {q_hat:,.2f}")
print()

if coverage >= 0.89 and coverage <= 0.91:
    print("✓ Đạt độ bao phủ mục tiêu 90%")
else:
    print("✗ Chưa đạt độ bao phủ mục tiêu 90%")

# So sánh với giá trị trung bình
median_price = np.median(y_train)
print(f"\nSo sánh với giá trị trung bình:")
print(f"  Median price: ${median_price:,.0f}")
print(f"  MAE as % of median: {(val_mae/median_price)*100:.1f}%")
print(f"  RMSE as % of median: {(val_rmse/median_price)*100:.1f}%")
print(f"  Interval width as % of median: {(mean_width/median_price)*100:.1f}%")


KẾT QUẢ CUỐI CÙNG - SPLIT CONFORMAL PREDICTION
Mô hình: Optimized Ensemble (LGBM + HistGB)
Số lượng features: 72
Số lượng mẫu test: 200,000

PERFORMANCE METRICS:
  Cross-Validation MAE: 56,514.96 ± 88.75
  Cross-Validation RMSE: 102,233.21 ± 1,142.42
  Training MAE: 43,880.18
  Training RMSE: 68,246.10
  Validation MAE: 56,072.10
  Validation RMSE: 102,032.00
  MAPE: 9.91%

CONFORMAL PREDICTION RESULTS:
  Coverage thực tế: 90.0%
  Average Winkler Score: 259,654.24
  Độ rộng khoảng trung bình: 257,249.33
  Quantile threshold (q_hat): 128,624.66

✓ Đạt độ bao phủ mục tiêu 90%

So sánh với giá trị trung bình:
  Median price: $460,000
  MAE as % of median: 12.2%
  RMSE as % of median: 22.2%
  Interval width as % of median: 55.9%


#### Xuất kết quả ra file CSV

In [15]:
test_results_df = pd.DataFrame({
    'prediction': test_pred,
    'pi_lower': pi_lower,
    'pi_upper': pi_upper,
    'interval_width': pi_upper - pi_lower
})

test_results_df.to_csv('models/optimized_test_predictions.csv', index=False)

# Tạo summary report
summary_report = {
    'Model': 'Optimized Ensemble (LGBM+HistGB) Split Conformal',
    'CV_MAE_Mean': np.mean(cv_mae_scores),
    'CV_MAE_Std': np.std(cv_mae_scores),
    'CV_RMSE_Mean': np.mean(cv_rmse_scores),
    'CV_RMSE_Std': np.std(cv_rmse_scores),
    'Train_MAE': train_mae,
    'Train_RMSE': train_rmse,
    'Val_MAE': val_mae,
    'Val_RMSE': val_rmse,
    'MAPE': mape,
    'Coverage': coverage,
    'Winkler_Score': avg_winkler,
    'Interval_Width': mean_width,
    'Q_Hat': q_hat
}

summary_df = pd.DataFrame([summary_report])
summary_df.to_csv('models/performance_summary.csv', index=False)

print("Đã xuất kết quả ra file:")
print("- models/optimized_test_predictions.csv")
print("- models/performance_summary.csv")
print("\nHoàn thành tối ưu hóa Split Conformal Prediction!")

Đã xuất kết quả ra file:
- models/optimized_test_predictions.csv
- models/performance_summary.csv

Hoàn thành tối ưu hóa Split Conformal Prediction!
